# Course project: cleavage plane for Ca doped YBCO

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from pwscf import make_struc_doped, make_struc_undoped, write_inputs, make_cleave_struc_undoped

## Cleave Z=2 Relaxed Cell

In [2]:
lattice_2_relax = np.array([
   [3.888243331,   0.000000000,   0.000000000],
   [0.000000000,   3.793282068,   0.000000000],
   [0.000000000,   0.000000000,  43.772261683]])

symbols_2_relax = ['Ba','Ba','Ba','Ba',
           'Y','Y',
           'Cu','Cu','Cu','Cu','Cu','Cu','Cu',
           'O','O','O','O','O','O','O','O','O','O','O','O','O','O','O']

pos_2_relax= np.array([[ 1.9441210480 ,       1.8966418432    ,    9.5363307681],
               [ 1.9441210480  ,      1.8966418432    ,   21.2364115021],
               [ 1.9441210480   ,     1.8966418432    ,    2.0684219014],
               [ 1.9441210480    ,    1.8966418432    ,   13.7685135537],
               [ 1.9441210480     ,   1.8966418432    ,    5.8085645285],
               [ 1.9441210480      ,  1.8966418432    ,   17.4962566006],
               [ 0.0000000000 ,       0.0000000000    ,    7.5500233693],
               [ 0.0000000000 ,       0.0000000000    ,   19.2238547829],
               [ 0.0000000000 ,       0.0000000000    ,    4.0810351969],
               [ 0.0000000000 ,       0.0000000000    ,   15.7549072365],
               [ 0.0000000000 ,       0.0000000000    ,    0.0839045299],
               [ 0.0000000000 ,       0.0000000000    ,   11.6524888709],
               [ 0.0000000000 ,       0.0000000000    ,   23.2208206425],
               [ 0.0000000000 ,       1.8966418432    ,   -0.1003135104],
               [ 0.0000000000 ,       1.8966418432    ,   11.6524690407],
               [ 0.0000000000 ,       1.8966418432    ,   23.4050526932],
               [ 1.9441210480 ,       0.0000000000    ,    7.2681752193],
               [ 1.9441210480 ,       0.0000000000    ,   18.9572038231],
               [ 1.9441210480 ,       0.0000000000    ,    4.3476717804],
               [ 1.9441210480 ,       0.0000000000    ,   16.0367037430],
               [ 0.0000000000 ,       1.8966418432    ,    7.1922414659],
               [ 0.0000000000 ,       1.8966418432    ,   18.9215511152],
               [ 0.0000000000 ,       1.8966418432    ,    4.3832718072],
               [ 0.0000000000 ,       1.8966418432    ,   16.1125728669],
               [ 0.0000000000 ,       0.0000000000    ,    9.7608360006],
               [ 0.0000000000 ,       0.0000000000    ,   21.3322842147],
               [ 0.0000000000 ,       0.0000000000    ,    1.9725514011],
               [ 0.0000000000 ,       0.0000000000    ,   13.544034172]])

sort_idx_2_relax = np.argsort(pos_2_relax[:,2])

In [3]:
[structure, name] = make_cleave_struc_undoped(lattice_2_relax, [symbols_2_relax[idx] for idx in sort_idx_2_relax], 
                          pos_2_relax[sort_idx_2_relax], 2, cleave_plane = 'Y', separation=10)
#print(name)
write_inputs(ecut = 60, nkxy = 8, nkz = 1, struc = structure, dirname = name, calc = 'vc-relax')

MAKING FILE
Writing /home/bond/Work/Test/YBCO_Y_cleave_10_sep.in


TextFile {'path': '/home/bond/Work/Test/YBCO_Y_cleave_10_sep.in', 'text': "&CONTROL\n    calculation = 'vc-relax'\n    pseudo_dir = '/n/holyscratch01/hoffman_lab/ruizhe/YBCO_Project/pseudo'\n    outdir = 'n/holyscratch01/hoffman_lab/ruizhe/outdir'\n    tstress = .true.\n    tprnfor = .true.\n    disk_io = 'none'\n    nstep = 300\n    etot_conv_thr = 1e-05\n    forc_conv_thr = 0.0001\n/ \n&SYSTEM\n    ecutwfc = 60\n    ecutrho = 720\n    occupations = 'smearing'\n    smearing = 'gaussian'\n    degauss = 0.01\n    ntyp = 4\n    nat = 28\n    ibrav = 0\n/ \n&ELECTRONS\n    diagonalization = 'david'\n    electron_maxstep = 120\n    mixing_mode = 'local-TF'\n    mixing_beta = 0.2\n    mixing_ndim = 10\n    conv_thr = 1e-06\n/ \n&IONS\n    ion_dynamics = 'bfgs'\n/ \n&CELL\n    cell_dynamics = 'bfgs'\n/ \nK_POINTS {automatic}\n 8 8 1  0 0 0\nATOMIC_SPECIES\n  Ba 137.327 Ba.pbe-nsp-van.UPF\n  Cu 63.546 Cu.pbe-n-van_ak.UPF\n  O 15.999 O.pbe-van_ak.UPF\n  Y 88.90584 Y.pbe-nsp-van.UPF\nCELL_PARAM

In [3]:
separations = np.linspace(1,8,8)
for separation in separations:
    for cleave_plane in ['BaO','CuO','Y']:
        [structure, name] = make_cleave_struc_undoped(lattice_2_relax, [symbols_2_relax[idx] for idx in sort_idx_2_relax], 
                          pos_2_relax[sort_idx_2_relax], 2, cleave_plane = cleave_plane, separation=separation)
        write_inputs(ecut = 60, nkxy = 8, nkz = 1, struc = structure, dirname = name, calc = 'vc-relax')

MAKING FILE
Writing /home/bond/Work/Input_files/YBCO_BaO_cleave_1.0_sep.in
MAKING FILE
Writing /home/bond/Work/Input_files/YBCO_CuO_cleave_1.0_sep.in
MAKING FILE
Writing /home/bond/Work/Input_files/YBCO_Y_cleave_1.0_sep.in
MAKING FILE
Writing /home/bond/Work/Input_files/YBCO_BaO_cleave_2.0_sep.in
MAKING FILE
Writing /home/bond/Work/Input_files/YBCO_CuO_cleave_2.0_sep.in
MAKING FILE
Writing /home/bond/Work/Input_files/YBCO_Y_cleave_2.0_sep.in
MAKING FILE
Writing /home/bond/Work/Input_files/YBCO_BaO_cleave_3.0_sep.in
MAKING FILE
Writing /home/bond/Work/Input_files/YBCO_CuO_cleave_3.0_sep.in
MAKING FILE
Writing /home/bond/Work/Input_files/YBCO_Y_cleave_3.0_sep.in
MAKING FILE
Writing /home/bond/Work/Input_files/YBCO_BaO_cleave_4.0_sep.in
MAKING FILE
Writing /home/bond/Work/Input_files/YBCO_CuO_cleave_4.0_sep.in
MAKING FILE
Writing /home/bond/Work/Input_files/YBCO_Y_cleave_4.0_sep.in
MAKING FILE
Writing /home/bond/Work/Input_files/YBCO_BaO_cleave_5.0_sep.in
MAKING FILE
Writing /home/bond/Wo

## Relaxed Undoped Z=3

In [5]:
lattice_3_relax =np.array([[3.864431295,   0.000016036,  -0.000000526],
   [0.000006690,   3.817298207,  -0.000006353],
  [-0.000008276,  -0.000093877,  55.034164756]])

symbols_3_relax= ['Ba','Ba','Ba','Ba','Ba','Ba',
                 'Y','Y','Y',
                 'Cu','Cu','Cu','Cu',
                  'Cu','Cu','Cu','Cu','Cu','Cu',
                 'O','O','O','O','O',
                 'O','O','O','O','O',
                 'O','O','O','O','O',
                 'O','O','O','O','O',
                 'O','O']

pos_3_relax=np.array([[     1.9679456186   ,     1.9290906912  ,      9.5574650775],
        [   1.9792024503     ,   1.9289399376    ,   21.3205595193],
        [    1.9521913659    ,    1.9292862521   ,    33.1114652714],
        [    1.9521506617    ,    1.9292515200   ,     2.0805219653],
        [    1.9791827347    ,    1.9289212038   ,    13.8707544476],
        [    1.9679854071    ,    1.9291242361   ,    25.6340937027],
        [     1.9554360864   ,     1.9291653018  ,      5.8391444224],
        [     1.9710999057   ,     1.9288623791  ,     17.5956280181],
        [     1.9554844916   ,     1.9291913903  ,     29.3526520398],
        [    0.0290459267    ,    0.0204755144   ,     7.5728170661],
        [    0.0426813668    ,    0.0201946470   ,    19.3373313559],
        [    0.0215634996    ,    0.0205710675   ,    31.0822157956],
        [    0.0215186020    ,    0.0205492854   ,     4.1097078606],
        [    0.0426801815    ,    0.0201852713   ,    15.8539162263],
        [    0.0290937929    ,    0.0205057344   ,    27.6189077760],
        [    0.0210617219    ,    0.0206344626   ,     0.1095296715],
        [   -0.0279921326    ,    0.0202712702   ,    11.7101428574],
        [   -0.0280618883    ,    0.0203058530   ,    23.4812697780],
        [    0.0211352185    ,    0.0206727862   ,    35.0825922110],
        [     0.0228984827   ,     1.9294118661  ,     -0.0391675108],
        [    -0.3302243943   ,     1.9289440511  ,     11.7085955828],
        [    -0.3304193626   ,     1.9289780916  ,     23.4828564845],
        [     0.0230047179   ,     1.9294591623  ,     35.2313082573],
        [     1.9603774394   ,     0.0206611310  ,      7.2829150360],
        [     1.9737812853   ,     0.0203590425  ,     19.0445794566],
        [     1.9537731333   ,     0.0207446927  ,     30.8033906594],
        [     1.9537248395   ,     0.0207186986  ,      4.3884619974],
        [     1.9737764867   ,     0.0203546058  ,     16.1466631759],
        [     1.9604194345   ,     0.0206891967  ,     27.9088452228],
        [     0.0223443427   ,     1.9291373377  ,      7.2466228150],
        [     0.0366303639   ,     1.9288688990  ,     19.0178774317],
        [     0.0221386910   ,     1.9292540261  ,     30.7878462581],
        [     0.0220866250   ,     1.9292313099  ,      4.4039980222],
        [     0.0366253134   ,     1.9288598837  ,     16.1733876733],
        [     0.0223902874   ,     1.9291697792  ,     27.9451323883],
        [     0.0812074626   ,     0.0204115869  ,      9.8244602123],
        [     0.0880851621   ,     0.0203556304  ,     21.5976783934],
        [     0.0199638079   ,     0.0206595764  ,     33.1962554542],
        [     0.0199442882   ,     0.0206341296  ,      1.9958799634],
        [     0.0881155600   ,     0.0203269321  ,     13.5937005745],
        [    0.0812258362    ,    0.0204542060   ,    25.3669848023]])

sort_idx_3_relax = np.argsort(pos_3_relax[:,2])

In [6]:
separations = np.linspace(1,6,6)
for separation in separations:
    for cleave_plane in ['BaO','CuO','Y']:
        make_cleave_struc_undoped(lattice_3_relax, [symbols_3_relax[idx] for idx in sort_idx_3_relax], 
                          pos_3_relax[sort_idx_3_relax], 3, cleave_plane = cleave_plane, separation=separation)
        ##make the input file

## Z Test

Make input files (from z-1 to z-4)

In [5]:
zs = np.arange(1,5)
for z in zs:
    [struc, struc_name] = make_struc_undoped(nxy = 1, nz = z, vacuum = 40)
    infile = write_inputs(struc = struc, dirname = 'LL_test', name = struc_name, calc = 'vc-relax')

MAKING FILE
Writing LL_test/YBCO_conv_111_40vac_NOcleave_0sep_vc-relax.in
MAKING FILE
Writing LL_test/YBCO_conv_112_40vac_NOcleave_0sep_vc-relax.in
MAKING FILE
Writing LL_test/YBCO_conv_113_40vac_NOcleave_0sep_vc-relax.in
MAKING FILE
Writing LL_test/YBCO_conv_114_40vac_NOcleave_0sep_vc-relax.in


## Separation Test

In [4]:
separation = 10
nz=3

a=make_struc_undoped(vacuum = 20, nz = nz, separation = separation, cleave_plane = 'Y', nxy= 2)
a=make_struc_undoped(vacuum = 20, nz = nz, separation = separation, cleave_plane = 'BaO')
a=make_struc_undoped(vacuum = 20, nz = nz, separation = separation, cleave_plane = 'CuO')

## Make Doped Cell

In [5]:
[struc, struc_name]= make_struc_doped(nz = 1, slab=False)
infile = write_inputs(struc = struc, dirname = struc_name, calc = 'vc-relax')

MAKING FILE
Writing /home/bond/Work/Input_files/dYBCO_rt2_111_0vac_NOcleave_0sep.in


In [3]:
struc_name

'dYBCO_rt2_111_0vac_NOcleave_0sep'

## Doped Separation Test

To make the cleaving plane on the doped sample I will need to sort all of the atoms by z height first, since they are not currently sorted within the 2xrt(2) supercell.

Maybe get atoms, sort by z, return indexes, sort chemical symbols by same indexes, then count up how many planes should be included for each cleaving.

In [48]:
nxy=1
nz = 1
alat=3.82 
blat=3.89 
clat=11.68
vacuum=0
cleave_plane='NO'
separation=0
slab = True

a = numpy.sqrt(alat**2 + blat**2)
lattice = numpy.array([[a,0,0],[0,a,0],[0,0,clat]])
symbols = ['Cu', 'Cu', 'O', 'O',
           'O','O','Ba', 'Ba',
           'Cu', 'Cu', 'O', 'O', 'O', 'O',
           'Y', 'Y',
           'O', 'O', 'O', 'O' ,'Cu', 'Cu',
           'Ba', 'Ba', 'O', 'O']
sc_pos = [[0,0,0], #Cu
          [0.5,0.5,0], #Cu
          [0.25,0.25,0], #O
          [0.75,0.75,0], #O
          [0,0,0.15918], #O
          [0.5,0.5,0.15918], #O
          [0.5, 0, 0.18061], #Ba
          [0, 0.5, 0.18061], #Ba
          [0,0,0.35332], #Cu
          [0.5,0.5,0.35332], #Cu
          [0.25,0.25,0.37835], #O
          [0.75,0.75,0.37835], #O
          [0.25,0.75,0.37935], #O
          [0.75,0.25,0.37935], #O
          [0.5,0,0.5], #Y
          [0,0.5,0.5], #Y
          [0.25,0.25,0.62065], #O
          [0.75,0.75,0.62065], #O
          [0.25,0.75,0.62165], #O
          [0.75,0.25,0.62165], #O
          [0,0,0.64668], #Cu
          [0.5,0.5,0.64668], #Cu
          [0.5, 0, 0.81939], #Ba
          [0, 0.5, 0.81939], #Ba
          [0,0,0.84082], #O
          [0.5,0.5,0.84082] #O
         ]
YBCO = Atoms(symbols=symbols, scaled_positions=sc_pos, cell=lattice)

#make an x/y supercell of 2 and dope 1/8 Y --> Ca
multiplier = numpy.identity(3)
multiplier[0,0]=2
multiplier[1,1]=2
supercell = make_supercell(YBCO, multiplier)

temp_sym = supercell.get_chemical_symbols()
temp_sym[15] = 'Ca'
supercell.set_chemical_symbols(temp_sym)
    

#make the supercell of the 2rt(2) doped supercell in the z direction
multiplier = numpy.identity(3)
multiplier[2,2]=nz
supercell = make_supercell(supercell, multiplier)

#sort the atoms so cleave plane can easily be determined
pos = supercell.get_positions()
sym = supercell.get_chemical_symbols()
args = numpy.argsort(pos[:,2])
supercell.set_positions(pos[args])
supercell.set_chemical_symbols([sym[j] for j in args])


if slab:
    #make the position of the 'capping' layer
    Cu_layer = Atoms(symbols = ['Cu', 'Cu', 'O', 'O'], 
                     scaled_positions = [[0.0,0.0, 0.0],
                                  [0.5,0.5,0],
                                  [0.25,0.25,0], 
                                  [0.75,0.75,0]],
                   cell = numpy.array([[a,0,0],[0,a,0],[0,0,clat]]))
    #make a supercell of the capping layer
    multiplier = numpy.identity(3)
    multiplier[0,0]=2
    multiplier[1,1]=2
    Cu_layer = make_supercell(Cu_layer, multiplier)

    #cap the unit cell
    supercell = stack(supercell, Cu_layer)

    #add vacuum
    add_vacuum(supercell, vacuum)

#find the plane to cleave on (closest to the middle)
    if cleave_plane == 'CuO':
        split = int(nz/2)*104 + 16
        
    elif cleave_plane == "BaO":
        split = int(nz/2)*104 + 32
        
    elif cleave_plane == "Y":
        split = int(nz/2)*104 + 64
    
    if cleave_plane != "NO":
        temp_pos = supercell.get_positions()
        temp_pos[split:,2] += separation #add separation in z to all atoms after cleave plane
        supercell.set_positions(temp_pos)
        temp_cell = supercell.get_cell()
        temp_cell[2][2] += separation #add separation to cell height so vacuum is unchanged
        supercell.set_cell(temp_cell)
        
#make the supercell of the 2rt(2) doped supercell in x/y
multiplier = numpy.identity(3)
multiplier[0,0]=nxy
multiplier[1,1]=nxy
supercell = make_supercell(supercell, multiplier)

#output to cif
name = f'YBCO_rt2_{nxy}{nxy}{nz}_{vacuum}vac_{cleave_plane}cleave_{separation}sep'
write(f'{name}.cif', supercell)

# FIx Doped CU

In [7]:
import os, copy, numpy
from labutil.objects import TextFile, ExternalCode, File, Param, Struc, ase2struc, Dir, Kpoints, PseudoPotential
from labutil.util import prepare_dir, run_command
from ase.spacegroup import crystal
from ase.io import write
from ase.build import bulk, make_supercell, add_vacuum, stack
from ase import Atoms

def make_struc_doped(nxy=1, nz = 2, alat=3.78, blat=3.88, clat=11.68, vacuum=0, cleave_plane='NO',
                         separation=0, slab = True):
    """
    Creates the crystal structure using ASE and saves to a cif file. Constructs a root2xroot2 YBCO structure
    with 1/8 Y --> Ca doping
    nxy, nz: unit cell dimensions follow  nxy *2root 2, nxy* 2root 2, nz 
    alat, blat, clat: conventianal (NOT root2) lattice parameters
    vacuum: vacuum spacing between slabs
    cleave_plane: Not yet implemented
    separation: not yet implemented
    slab: if true add a CuO capping layer
    :return: structure object converted from ase
    
    Slab will be 'capped' with a CuO layer (will not make sense in bulk) """
    
    a = numpy.sqrt(alat**2 + blat**2)
    lattice = numpy.array([[-alat,blat,0],[alat,blat,0],[0,0,clat]])
    symbols = ['Cu', 'Cu', 'O', 'O',
               'O','O','Ba', 'Ba',
               'Cu', 'Cu', 'O', 'O', 'O', 'O',
               'Y', 'Y',
               'O', 'O', 'O', 'O' ,'Cu', 'Cu',
               'Ba', 'Ba', 'O', 'O']
    sc_pos = [[0,0,0], #Cu
              [0.5,0.5,0], #Cu
              [0.25,0.25,0], #O
              [0.75,0.75,0], #O
              [0,0,0.15918], #O
              [0.5,0.5,0.15918], #O
              [0.5, 0, 0.18061], #Ba
              [0, 0.5, 0.18061], #Ba
              [0,0,0.35332], #Cu
              [0.5,0.5,0.35332], #Cu
              [0.25,0.25,0.37835], #O
              [0.75,0.75,0.37835], #O
              [0.25,0.75,0.37935], #O
              [0.75,0.25,0.37935], #O
              [0.5,0,0.5], #Y
              [0,0.5,0.5], #Y
              [0.25,0.25,0.62065], #O
              [0.75,0.75,0.62065], #O
              [0.25,0.75,0.62165], #O
              [0.75,0.25,0.62165], #O
              [0,0,0.64668], #Cu
              [0.5,0.5,0.64668], #Cu
              [0.5, 0, 0.81939], #Ba
              [0, 0.5, 0.81939], #Ba
              [0,0,0.84082], #O
              [0.5,0.5,0.84082] #O
             ]
    YBCO = Atoms(symbols=symbols, scaled_positions=sc_pos, cell=lattice)
    
    #make an x/y supercell of 2 and dope 1/8 Y --> Ca
    multiplier = numpy.identity(3)
    multiplier[0,0]=2
    multiplier[1,1]=2
    supercell = make_supercell(YBCO, multiplier)
    
    temp_sym = supercell.get_chemical_symbols()
    temp_sym[15] = 'Ca'
    supercell.set_chemical_symbols(temp_sym)
    
    #make the supercell of the 2rt(2) doped supercell
    multiplier = numpy.identity(3)
    multiplier[0,0]=nxy
    multiplier[1,1]=nxy
    multiplier[2,2]=nz
    supercell = make_supercell(supercell, multiplier)
    
    
    if slab:
        #make the position of the 'capping' layer
        Cu_layer = Atoms(symbols = ['Cu', 'Cu', 'O', 'O'], 
                         scaled_positions = [[0.0,0.0, 0.0],
                                      [0.5,0.5,0],
                                      [0.25,0.25,0], 
                                      [0.75,0.75,0]],
                       cell = numpy.array([[a,0,0],[0,a,0],[0,0,clat]]))
        #make a supercell of the capping layer
        multiplier = numpy.identity(3)
        multiplier[0,0]=nxy*2
        multiplier[1,1]=nxy*2
        Cu_layer = make_supercell(Cu_layer, multiplier)

        #cap the unit cell
        supercell = stack(supercell, Cu_layer)

        #add vacuum
        add_vacuum(supercell, vacuum)
    
    #output to cif
    name = f'dYBCO_rt2_{nxy}{nxy}{nz}_{vacuum}vac_{cleave_plane}cleave_{separation}sep'
    write(f'{name}.cif', supercell)
    structure = Struc(ase2struc(supercell))
    
    return [structure, name]

In [8]:
[struc, struc_name]= make_struc_doped(nz = 1, slab=False)
infile = write_inputs(struc = struc, dirname = struc_name, calc = 'vc-relax')

MAKING FILE
Writing /home/bond/Work/Input_files/dYBCO_rt2_111_0vac_NOcleave_0sep.in


# Inital relax calculation (RUNS A CALC)

In [ ]:
def relax_YBCO(nk, ecut):
    pseudopots = {'Y': PseudoPotential(ptype='uspp', element='Y', functional='LDA', name='Y.pz-spn-rrkjus_psl.1.0.0.UPF'),
                  'Ba': PseudoPotential(ptype='uspp', element='Ba', functional='LDA', name='Ba.pz-spn-rrkjus_psl.1.0.0.UPF'),
                  'Cu': PseudoPotential(ptype='uspp', element='Cu', functional='LDA', name='Cu.pz-d-rrkjus.UPF'),
                  'O': PseudoPotential(ptype='uspp', element='O', functional='LDA', name='O.pz-rrkjus.UPF')}
    struc = make_struc_non_doped()
    kpts = Kpoints(gridsize=[nk, nk, nk], option='automatic', offset=True)
    dirname = 'YBCO_relax_ecut_{}_nk_{}'.format(ecut, nk)
    runpath = Dir(path=os.path.join(os.environ['WORKDIR'], "Project", dirname))
    input_params = PWscf_inparam({
        'CONTROL': {
            'calculation': 'relax',
            'pseudo_dir': os.environ['QE_POTENTIALS'],
            'outdir': runpath.path,
            'tstress': True,
            'tprnfor': True,
            'disk_io': 'none'
        },
        'SYSTEM': {
            'ecutwfc': ecut,
            'ecutrho': ecut * 12,
            'occupations': 'smearing',
            'smearing': 'mp',
            'degauss': 0.02
             },
        'ELECTRONS': {
            'diagonalization': 'david',
            'mixing_beta': 0.7,
            'conv_thr': 1e-7,
        },
        'IONS': {
            'ion_dynamics': 'bfgs'
        },
        'CELL': {},
        })

    output_file = run_qe_pwscf(runpath=runpath, struc=struc,  pseudopots=pseudopots,
                               params=input_params, kpoints=kpts, ncpu=16)
    output = parse_qe_pwscf_output(outfile=output_file)
    return output

def kpts_scan():
    ecut = 30
    nk = np.arange(1, 2)
    energy=[]
    for i in nk:
        output = relax_YBCO(ecut=ecut, nk=i)
        energy.append(output['energy'])
        print(i)
    return energy